http://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/  
https://www.w3schools.com/colors/colors_names.asp

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
class HTMLTableParser:
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table('id'), self.parse_html_table(table)) for table in soup.find_all('table')]
    def parse_html_table(self, table):
        n_columns = 0
        n_rows = 0
        column_names = []
        for row in table.find_all('tr'):
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows += 1
                if n_columns == 0:
                    n_columns = len(td_tags)
            th_tags = row.find_all('th')
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception('Column titles do not match the number of columns')

        columns = column_names if len(column_names) > 0 else range(0, n_columns)
        df = pd.DataFrame(columns=columns, index=range(0, n_rows))
        row_maker = 0
        for row in table.find_all('tr'):
            column_maker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_maker, column_maker] = column.get_text()
                column_maker += 1
            if len(columns) > 0:
                row_maker += 1
        for col in df:
                try:
                    df[col] = df[col].astype(float)
                except ValueError:
                    pass
            
        return df    

In [19]:
hp = HTMLTableParser()
table = hp.parse_url('https://www.w3schools.com/colors/colors_names.asp')[0][1]
table.head()

,Color Name,HEX,Color,Shades,Mix
0,AliceBlue,#F0F8FF,,Shades,Mix
1,AntiqueWhite,#FAEBD7,,Shades,Mix
2,Aqua,#00FFFF,,Shades,Mix
3,Aquamarine,#7FFFD4,,Shades,Mix
4,Azure,#F0FFFF,,Shades,Mix


In [31]:
import requests
url = 'http://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=421&aid=0002598201&date=20170308'
#url1 = 'https://docs.oracle.com/javase/7/docs/api/'
hp = HTMLTableParser()
table = hp.parse_url(url)[0][1]
table.head()

,0,1,2
0,\n\n랭킹뉴스\n\n많이 본 뉴스 선택됨\n주간 클릭 \n주간댓글 \n주간 검색어...,\n\n\r\n\tif (nil) {\r\n\t\tnil.init('naver_ne...,\n\n\n\n\n\n가장 많이 본 뉴스\n\n\n\n종합\n정치\n경제\n사회\n...
